# HW6 — FX Carry Strategy (GBP funding)  
**Objective:** Implement a weekly FX carry backtest using:
- **Funding leg:** UK OIS (IUDSOIA) + 50bp, applied to **4/5 of USD notional** (5x leverage proxy).  
- **Lending leg:** 5Y par bond in each EM currency with **quarterly coupons** at the **5Y swap rate**.  
- **Mark-to-market:** reprice the bond each week using the **new swap curve**, with time-to-cashflows reduced by **1/52 years**.  
- **Entry rule:** trade only if `s5_lend - s5_fund >= 50bp`.  
- **Accounting:** Convert all cashflows to **USD** (home currency).  

**Data locations (relative to this notebook):**
- EM swap curves + EM FX: `../../Data/`  
- UK OIS (IUDSOIA) + FX tables (Nasdaq EDI/CUR extracts): `./data_clean/`

**Primary reference notebook:** `Zero_And_Spot_Curves.ipynb` (bootstrapping/pricing patterns).  
**Concept reference:** `Carry_Concept__FTSE.pdf` (carry + rolldown intuition and diagnostics).

In [ ]:

# --- imports ---
from __future__ import annotations

import warnings
warnings.filterwarnings("ignore")

from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Iterable, Tuple, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:

# --- paths / constants ---
NB_DIR = Path.cwd()  # assume notebook is run from its own directory
DATA_EM_DIR = (NB_DIR / "../../Data").resolve()
DATA_CLEAN_DIR = (NB_DIR / "data_clean").resolve()

USD_NOTIONAL = 10_000_000.0
BORROW_FRAC = 0.80               # 4/5 notional borrowed
BORROW_SPREAD = 0.0050           # 50bp
WEEK_DT = 1.0 / 52.0
COUPON_FREQ = 4                  # quarterly
FUNDING_CCY = "GBP"              # per assignment name "FXCarryBasedOnGBP"

EM_CCYS = ["TRY", "ZAR", "PKR", "BRL", "NGN"]

print("NB_DIR        :", NB_DIR)
print("DATA_EM_DIR   :", DATA_EM_DIR, "exists:", DATA_EM_DIR.exists())
print("DATA_CLEAN_DIR:", DATA_CLEAN_DIR, "exists:", DATA_CLEAN_DIR.exists())

## 1) Data exploration (required)

We begin by inventorying inputs and inspecting their schema.  
The code below is written to be **path-robust**: it searches for plausible filenames and prints what it finds.

In [ ]:

def ls_tree(p: Path, max_items: int = 50) -> pd.DataFrame:
    if not p.exists():
        return pd.DataFrame({"path":[str(p)], "exists":[False]})
    items = sorted([x for x in p.rglob("*") if x.is_file()])
    items = items[:max_items]
    return pd.DataFrame({
        "relpath":[str(x.relative_to(p)) for x in items],
        "suffix":[x.suffix.lower() for x in items],
        "size_kb":[round(x.stat().st_size/1024,2) for x in items],
    })

display(ls_tree(DATA_CLEAN_DIR, 200))
display(ls_tree(DATA_EM_DIR, 200))

In [ ]:

def find_one(root: Path, patterns: Iterable[str]) -> Path:
    pats = list(patterns)
    hits = []
    for pat in pats:
        hits += list(root.rglob(pat))
    hits = [h for h in hits if h.is_file()]
    if not hits:
        raise FileNotFoundError(f"No files under {root} matching {pats}")
    hits = sorted(hits, key=lambda x: (len(str(x)), str(x)))
    return hits[0]

def read_table(path: Path) -> pd.DataFrame:
    suf = path.suffix.lower()
    if suf == ".csv":
        return pd.read_csv(path)
    if suf == ".parquet":
        return pd.read_parquet(path)
    if suf in {".xlsx", ".xls"}:
        return pd.read_excel(path)
    raise ValueError(f"Unsupported file type: {path}")

## 2) Load UK OIS (IUDSOIA)

We need **short-tenor UK OIS** (daily), used as the weekly funding rate input:
`r_borrow = OIS + 50bp`.  

Expected columns vary; we standardize to:
- `date` (datetime64[ns], tz-naive)
- `ois` (decimal, e.g. 0.0425 = 4.25%)

In [ ]:

# Locate IUDSOIA
iudsoia_path = find_one(DATA_CLEAN_DIR, patterns=["*IUDSOIA*.csv", "*IUDSOIA*.parquet", "*iudsoia*.csv", "*iudsoia*.parquet"])
print("IUDSOIA file:", iudsoia_path)

ois_raw = read_table(iudsoia_path)
display(ois_raw.head())
print(ois_raw.columns)

In [ ]:

def standardize_ois(df: pd.DataFrame) -> pd.Series:
    d = df.copy()
    date_col = next((c for c in d.columns if str(c).lower() in {"date","dt","time","observation_date"}), None)
    if date_col is None:
        date_col = d.columns[0]
    d[date_col] = pd.to_datetime(d[date_col])
    d = d.sort_values(date_col)

    rate_col = next((c for c in d.columns if str(c).lower() in {"ois","iudsoia","value","rate"}), None)
    if rate_col is None:
        num_cols = [c for c in d.columns if c != date_col and pd.api.types.is_numeric_dtype(d[c])]
        if not num_cols:
            raise ValueError("Cannot infer OIS rate column.")
        rate_col = num_cols[0]

    s = d.set_index(date_col)[rate_col].astype(float)
    if s.dropna().median() > 1.0:
        s = s / 100.0
    s.name = "ois"
    return s

ois = standardize_ois(ois_raw)
ois = ois[~ois.index.duplicated(keep="last")].sort_index()
display(ois.head())
display(ois.tail())

## 3) Load FX rates (Nasdaq EDI/CUR extracts)

We need:
- USD/GBP (funding conversion)
- USD per EM currency (preferred), or the inverse (we will standardize)

Exports differ; we support:
- wide table: `date, GBP, TRY, ...`
- long table: `date, code, value`

We standardize to a wide DataFrame: `fx_usd_per[CCY] = USD per 1 unit of CCY`.

In [ ]:

fx_path = find_one(DATA_CLEAN_DIR, patterns=["*EDI*CUR*.csv", "*edi*cur*.csv", "*fx*.csv", "*FX*.csv", "*currency*.csv", "*CUR*.csv"])
print("FX file:", fx_path)

fx_raw = read_table(fx_path)
display(fx_raw.head())
print(fx_raw.columns)

In [ ]:

def standardize_fx_to_usd_per_ccy(df: pd.DataFrame, ccys: Iterable[str]) -> pd.DataFrame:
    ccys = [c.upper() for c in ccys]
    d = df.copy()

    date_col = next((c for c in d.columns if str(c).lower() in {"date","dt","time","timestamp"}), None)
    if date_col is None:
        date_col = d.columns[0]
    d[date_col] = pd.to_datetime(d[date_col])

    wide_cols = set(map(str.upper, d.columns))
    if all(c in wide_cols for c in ccys):
        out = d[[date_col] + ccys].copy()
        out = out.rename(columns={date_col:"date"}).set_index("date").sort_index()
        out.columns = [c.upper() for c in out.columns]
    else:
        code_col = next((c for c in d.columns if str(c).lower() in {"code","ccy","currency","symbol","cur"}), None)
        val_col  = next((c for c in d.columns if str(c).lower() in {"value","rate","px","price","close","mid"}), None)
        if code_col is None or val_col is None:
            raise ValueError("Cannot infer FX long-form columns (need code and value).")
        out = (d[[date_col, code_col, val_col]]
               .rename(columns={date_col:"date", code_col:"ccy", val_col:"value"}))
        out["ccy"] = out["ccy"].astype(str).str.upper()
        out = out[out["ccy"].isin(ccys)]
        out = out.pivot_table(index="date", columns="ccy", values="value", aggfunc="last").sort_index()

    out = out.astype(float)

    # Orientation heuristic using GBP: USD/GBP typically O(1), not O(10^1-10^2)
    if "GBP" in out.columns:
        med = out["GBP"].dropna().median()
        if med > 10:
            out["GBP"] = 1.0 / out["GBP"]

    return out

FX_CCYS = [FUNDING_CCY] + EM_CCYS
fx_usd_per = standardize_fx_to_usd_per_ccy(fx_raw, FX_CCYS)
display(fx_usd_per.tail())

## 4) Load swap curves (Emerging Mkt YC)

We need (at minimum) **1Y and 5Y** par swap rates for each EM currency (more tenors is better).

We standardize to long form:
- `date`, `ccy`, `tenor` (years), `par_rate` (decimal)

In [ ]:

em_yc_path = find_one(DATA_EM_DIR, patterns=["*Emerging*Mkt*YC*.csv", "*Emerging*Mkt*YC*.parquet", "*Mkt*YC*.csv", "*Mkt*YC*.parquet", "*swap*curve*.csv", "*swap*curve*.parquet"])
print("EM YC file:", em_yc_path)

em_raw = read_table(em_yc_path)
display(em_raw.head())
print(em_raw.columns)

In [ ]:

import re

def standardize_swap_curve_long(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()

    date_col = next((c for c in d.columns if str(c).lower() in {"date","dt","time","asof"}), None)
    if date_col is None:
        date_col = d.columns[0]
    d[date_col] = pd.to_datetime(d[date_col])

    ccy_col = next((c for c in d.columns if str(c).lower() in {"ccy","currency","cur","code"}), None)
    tenor_col = next((c for c in d.columns if str(c).lower() in {"tenor","mat","maturity","years","t"}), None)
    rate_col  = next((c for c in d.columns if str(c).lower() in {"rate","swap","par_rate","y","yield","value"}), None)

    if ccy_col and tenor_col and rate_col:
        out = d[[date_col, ccy_col, tenor_col, rate_col]].rename(columns={
            date_col:"date", ccy_col:"ccy", tenor_col:"tenor", rate_col:"par_rate"
        })
    else:
        out_rows = []
        for col in d.columns:
            if col == date_col:
                continue
            s = str(col).upper()
            m = re.match(r"(?P<ccy>[A-Z]{3})[_\\-\\s]?(?P<ten>\\d+(\\.\\d+)?)\\s*(Y|YR|YEAR|YEARS)?$", s)
            if not m:
                m = re.match(r"(?P<ccy>[A-Z]{3})(?P<ten>\\d+(\\.\\d+)?)(Y|YR|YEAR|YEARS)$", s)
            if not m:
                continue
            ccy = m.group("ccy")
            ten = float(m.group("ten"))
            out_rows.append(pd.DataFrame({"date": d[date_col], "ccy": ccy, "tenor": ten, "par_rate": d[col]}))
        if not out_rows:
            raise ValueError("Cannot infer swap curve structure (need long form or parsable wide columns).")
        out = pd.concat(out_rows, ignore_index=True)

    out["ccy"] = out["ccy"].astype(str).str.upper()
    out["tenor"] = out["tenor"].astype(float)
    out["par_rate"] = out["par_rate"].astype(float)

    if out["par_rate"].dropna().median() > 1.0:
        out["par_rate"] = out["par_rate"] / 100.0

    out = out.dropna(subset=["date","ccy","tenor","par_rate"]).sort_values(["ccy","date","tenor"])
    return out

swap_long = standardize_swap_curve_long(em_raw)
swap_long = swap_long[swap_long["ccy"].isin(EM_CCYS + [FUNDING_CCY])]
display(swap_long.head())
print("ccys:", swap_long["ccy"].unique())

## 5) Bootstrapping + bond pricing (generalized from Zero_And_Spot_Curves)

We implement a **par-curve → zero-curve** bootstrap and a **quarterly coupon bond pricer**.

In [ ]:

def _interp_z(times: np.ndarray, z_tenors: np.ndarray, z_rates: np.ndarray) -> np.ndarray:
    return np.interp(times, z_tenors, z_rates, left=z_rates[0], right=z_rates[-1])

def zcb_from_par_curve(par: pd.Series, freq: int = COUPON_FREQ) -> pd.Series:
    '''
    Bootstrap continuous-compounded zero rates at the par curve's tenor pillars.
    Input: par[tenor_years] = par coupon rate (decimal) for a par bond with coupon freq.
    Output: zcb[tenor_years] = continuously-compounded zero rate at that tenor.
    '''
    par = par.dropna().sort_index()
    tenors = par.index.to_numpy(dtype=float)
    z = pd.Series(index=tenors, dtype=float)

    delta = 1.0 / freq

    for T in tenors:
        c = float(par.loc[T])
        times = np.arange(delta, T + 1e-12, delta)
        if len(times) == 0:
            z.loc[T] = np.nan
            continue
        times_pre = times[:-1]
        if len(times_pre) == 0:
            D_T = 1.0 / (1.0 + c*delta)
        else:
            known_t = z.dropna().index.to_numpy(dtype=float)
            known_z = z.dropna().to_numpy(dtype=float)
            if len(known_t) == 0:
                z_pre = np.full_like(times_pre, c)
            else:
                z_pre = _interp_z(times_pre, known_t, known_z)
            D_pre = np.exp(-z_pre * times_pre)
            pv_cpn_pre = (c*delta) * D_pre.sum()
            D_T = (1.0 - pv_cpn_pre) / (1.0 + c*delta)

        D_T = float(np.clip(D_T, 1e-12, 1.0))
        z.loc[T] = -np.log(D_T) / T

    return z

def bond_price_from_zcb(zcb: pd.Series, coupon_rate: float, tenor: float, freq: int = COUPON_FREQ) -> float:
    '''
    Clean PV per 1 notional for a fixed-rate bond with coupon_rate (annual, decimal),
    coupons at freq, maturity tenor (years), discounted by zcb curve (cc zero rates).
    '''
    delta = 1.0 / freq
    times = np.arange(delta, tenor + 1e-12, delta)
    if len(times) == 0:
        return 1.0
    z_t = zcb.index.to_numpy(dtype=float)
    z_r = zcb.to_numpy(dtype=float)
    z_i = _interp_z(times, z_t, z_r)
    D = np.exp(-z_i * times)
    pv_cpn = (coupon_rate * delta) * D.sum()
    pv_pri = D[-1]
    return float(pv_cpn + pv_pri)

def price_weekly_roll(par_curve_new: pd.Series, coupon_rate_orig: float, tenor_orig: float = 5.0,
                      dt: float = WEEK_DT, freq: int = COUPON_FREQ) -> float:
    rem = max(tenor_orig - dt, 1e-6)
    zcb_new = zcb_from_par_curve(par_curve_new, freq=freq)
    return bond_price_from_zcb(zcb_new, coupon_rate=coupon_rate_orig, tenor=rem, freq=freq)

# sanity check
_par = pd.Series({1.0:0.04, 2.0:0.042, 3.0:0.043, 5.0:0.045, 10.0:0.047})
_zcb = zcb_from_par_curve(_par)
p0 = bond_price_from_zcb(_zcb, coupon_rate=_par[5.0], tenor=5.0)
print("Par bond price (should be ~1):", p0)

## 6) Curve panels + weekly alignment

In [ ]:

def curve_panel(swap_long: pd.DataFrame, ccy: str) -> pd.DataFrame:
    d = swap_long[swap_long["ccy"] == ccy].copy()
    if d.empty:
        raise ValueError(f"No curve data for {ccy}.")
    piv = d.pivot_table(index="date", columns="tenor", values="par_rate", aggfunc="last").sort_index()
    piv.columns = piv.columns.astype(float)
    piv = piv.sort_index(axis=1)
    return piv

curves: Dict[str, pd.DataFrame] = {ccy: curve_panel(swap_long, ccy) for ccy in sorted(set(swap_long["ccy"]))}
print("curve currencies:", list(curves.keys()))

In [ ]:

def choose_weekly_dates(idx: pd.DatetimeIndex, weekday: int = 2, tol_days: int = 2) -> pd.DatetimeIndex:
    idx = pd.DatetimeIndex(idx).sort_values()
    start = idx.min().normalize()
    end = idx.max().normalize()
    anchors = pd.date_range(start=start, end=end, freq="W-" + ["MON","TUE","WED","THU","FRI","SAT","SUN"][weekday])

    chosen = []
    idx_set = set(idx)
    for a in anchors:
        window = [a + pd.Timedelta(days=k) for k in range(0, tol_days+1)] + \
                 [a - pd.Timedelta(days=k) for k in range(1, tol_days+1)]
        pick = next((d for d in window if d in idx_set), None)
        if pick is not None:
            chosen.append(pick)
    return pd.DatetimeIndex(sorted(set(chosen)))

def align_weekly_inputs(curves: Dict[str, pd.DataFrame], ois: pd.Series, fx_usd_per: pd.DataFrame,
                        weekday: int = 2, tol_days: int = 2):
    idx = ois.index
    for _,df in curves.items():
        idx = idx.intersection(df.index)
    idx = idx.intersection(fx_usd_per.index)

    if len(idx) == 0:
        raise ValueError("No overlapping dates across OIS, FX, and curves. Check parsing/sources.")

    wk = choose_weekly_dates(idx, weekday=weekday, tol_days=tol_days)

    ois_d = ois.reindex(idx).ffill().loc[wk]
    fx_d  = fx_usd_per.reindex(idx).ffill().loc[wk]
    curves_w = {c: df.reindex(idx).ffill().loc[wk] for c,df in curves.items()}
    return wk, curves_w, ois_d, fx_d

wk_dates, curves_w, ois_w, fx_w = align_weekly_inputs(curves, ois, fx_usd_per)
print("weekly obs:", len(wk_dates), "from", wk_dates.min().date(), "to", wk_dates.max().date())
display(fx_w.head())

## 7) Weekly P&L per currency (core deliverable)

In [ ]:

def get_s5(curve_row: pd.Series) -> float:
    ten = curve_row.index.to_numpy(dtype=float)
    val = curve_row.to_numpy(dtype=float)
    return float(np.interp(5.0, ten, val))

def weekly_pnl_one_ccy(
    lend_ccy: str,
    fund_ccy: str,
    curves_w: Dict[str, pd.DataFrame],
    ois_w: pd.Series,
    fx_w: pd.DataFrame,
    notional_usd: float = USD_NOTIONAL,
    borrow_frac: float = BORROW_FRAC,
    borrow_spread: float = BORROW_SPREAD,
    dt: float = WEEK_DT,
    freq: int = COUPON_FREQ,
) -> pd.DataFrame:
    if fund_ccy not in fx_w.columns or lend_ccy not in fx_w.columns:
        raise KeyError(f"FX missing for {fund_ccy} or {lend_ccy}. Columns={list(fx_w.columns)}")

    fund_curve = curves_w.get(fund_ccy, None)
    use_fund_curve = fund_curve is not None

    rows = []
    for i in range(len(fx_w.index)-1):
        t0 = fx_w.index[i]
        t1 = fx_w.index[i+1]

        fx_f0, fx_f1 = float(fx_w.loc[t0, fund_ccy]), float(fx_w.loc[t1, fund_ccy])
        fx_l0, fx_l1 = float(fx_w.loc[t0, lend_ccy]), float(fx_w.loc[t1, lend_ccy])

        lend0 = curves_w[lend_ccy].loc[t0].dropna()
        lend1 = curves_w[lend_ccy].loc[t1].dropna()
        if len(lend0) < 2 or len(lend1) < 2:
            continue

        s5_l = get_s5(lend0)
        if use_fund_curve:
            fund0 = fund_curve.loc[t0].dropna()
            s5_f = get_s5(fund0) if len(fund0) >= 2 else float(ois_w.loc[t0])
        else:
            s5_f = float(ois_w.loc[t0])

        active = (s5_l - s5_f) >= 0.0050
        if not active:
            rows.append((t1, 0.0, 0.0, 0.0, 0.0, s5_l, s5_f, False))
            continue

        # lending leg
        N_lend = notional_usd / fx_l0
        p1 = price_weekly_roll(lend1, coupon_rate_orig=s5_l, tenor_orig=5.0, dt=dt, freq=freq)
        cpn_accr_lend = N_lend * s5_l * dt
        lend_value_usd_t1 = (N_lend * p1 + cpn_accr_lend) * fx_l1
        lend_pnl = lend_value_usd_t1 - notional_usd

        # funding leg
        borrow_usd = borrow_frac * notional_usd
        B_fund = borrow_usd / fx_f0
        r_borrow = float(ois_w.loc[t0]) + borrow_spread
        int_fund = B_fund * r_borrow * dt
        repay_usd_t1 = (B_fund + int_fund) * fx_f1
        fund_pnl = borrow_usd - repay_usd_t1

        total_pnl = lend_pnl + fund_pnl
        ret = total_pnl / notional_usd
        rows.append((t1, total_pnl, ret, lend_pnl, fund_pnl, s5_l, s5_f, True))

    out = pd.DataFrame(rows, columns=["date","pnl_usd","ret","lend_pnl_usd","fund_pnl_usd","s5_lend","s5_fund","active"]).set_index("date")
    return out

pnl_panels = {ccy: weekly_pnl_one_ccy(ccy, FUNDING_CCY, curves_w, ois_w, fx_w) for ccy in EM_CCYS}
{k: v.shape for k,v in pnl_panels.items()}

## 8) Performance analysis (required)

In [ ]:

def max_drawdown(x: pd.Series) -> float:
    peak = x.cummax()
    dd = (x / peak) - 1.0
    return float(dd.min())

def ann_sharpe_weekly(r: pd.Series) -> float:
    r = r.dropna()
    if len(r) < 10 or r.std(ddof=1) == 0:
        return np.nan
    return float(np.sqrt(52.0) * r.mean() / r.std(ddof=1))

stats = []
rets = {}
for ccy, df in pnl_panels.items():
    r = df["ret"].fillna(0.0)
    rets[ccy] = r
    wealth = (1.0 + r).cumprod()
    stats.append({
        "ccy": ccy,
        "active_weeks": int(df["active"].sum()),
        "ann_sharpe": ann_sharpe_weekly(r),
        "ann_mean": float(52.0 * r.mean()),
        "ann_vol": float(np.sqrt(52.0) * r.std(ddof=1)),
        "max_dd": max_drawdown(wealth),
    })

stats_df = pd.DataFrame(stats).sort_values("ann_sharpe", ascending=False)
display(stats_df)

In [ ]:

rets_df = pd.DataFrame(rets).reindex(sorted(rets.keys()), axis=1)
corr = rets_df.corr()

plt.figure(figsize=(7,5))
plt.imshow(corr.values, aspect="auto")
plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha="right")
plt.yticks(range(len(corr.index)), corr.index)
plt.colorbar()
plt.title("Correlation of weekly carry returns (by lending currency)")
plt.tight_layout()
plt.show()

In [ ]:

active_df = pd.DataFrame({ccy: pnl_panels[ccy]["active"].astype(int) for ccy in EM_CCYS})
ret_df = pd.DataFrame({ccy: pnl_panels[ccy]["ret"].fillna(0.0) for ccy in EM_CCYS})

w = active_df.div(active_df.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)
port_ret = (w * ret_df).sum(axis=1)
port_wealth = (1.0 + port_ret).cumprod()

plt.figure(figsize=(8,4))
plt.plot(port_wealth.index, port_wealth.values)
plt.title("Equal-weight FX carry portfolio (weekly rebalanced; active-only weights)")
plt.ylabel("Wealth (start=1)")
plt.tight_layout()
plt.show()

print("Portfolio ann Sharpe:", ann_sharpe_weekly(port_ret))
print("Portfolio max drawdown:", max_drawdown(port_wealth))

## 9) Robustness checks (recommended)

In [ ]:

def run_sensitivity_weekday(weekday: int) -> dict:
    wk_dates2, curves_w2, ois_w2, fx_w2 = align_weekly_inputs(curves, ois, fx_usd_per, weekday=weekday, tol_days=2)
    pnl2 = {ccy: weekly_pnl_one_ccy(ccy, FUNDING_CCY, curves_w2, ois_w2, fx_w2) for ccy in EM_CCYS}
    active2 = pd.DataFrame({ccy: pnl2[ccy]["active"].astype(int) for ccy in EM_CCYS})
    ret2 = pd.DataFrame({ccy: pnl2[ccy]["ret"].fillna(0.0) for ccy in EM_CCYS})
    w2 = active2.div(active2.sum(axis=1).replace(0, np.nan), axis=0).fillna(0.0)
    pr = (w2 * ret2).sum(axis=1)
    wealth = (1.0 + pr).cumprod()
    return {"weekday":weekday, "ann_sharpe":ann_sharpe_weekly(pr), "max_dd":max_drawdown(wealth), "weeks":len(pr)}

pd.DataFrame([run_sensitivity_weekday(wd) for wd in [1,2,3]])  # Tue, Wed, Thu

## 10) Export outputs for report generation

In [ ]:

OUT_DIR = NB_DIR / "outputs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

stats_df.to_csv(OUT_DIR / "currency_stats.csv", index=False)
corr.to_csv(OUT_DIR / "currency_corr.csv")
port_ret.to_frame("port_ret").to_csv(OUT_DIR / "portfolio_weekly_returns.csv")

print("Wrote outputs to:", OUT_DIR)

### Notebook checklist

- [ ] Data files found under `../../Data/` and `./data_clean/`  
- [ ] Weekly aligned dataset has non-trivial history across all currencies  
- [ ] Per-currency weekly returns created  
- [ ] Correlation matrix + drawdowns computed  
- [ ] `./outputs/` contains tables for report generation